In [1]:
import getpass
import sys
import time
from collections import defaultdict

import pandas as pd
from scipy import stats
import numpy as np
from tqdm import tqdm


In [2]:
getpass.getuser()

'danieldomingo'

In [3]:
sys.version

'3.8.16 (default, Mar  1 2023, 21:19:10) \n[Clang 14.0.6 ]'

In [4]:
time.asctime()

'Mon Apr 17 14:20:36 2023'

# Get transcriptomic and target vectors

In [5]:
transcriptomic_responses_df = pd.read_csv(
    '../data/Transcriptional_data_frames/transcriptional_response_vectors_cell_line.tsv',
    sep='\t',
    index_col=0,
)

In [6]:
transcriptomic_responses_df.head(1)

,AAAS,AADAC,AADACL2,AADAT,AASS,AATF,ABAT,ABCA1,ABCB1,ABCB11,...,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN5DP,ZSCAN9,ZXDA,ZXDB,ZXDC,ZZZ3
CID00001_HA1E,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,-1,0,0,0,-1


In [7]:
#Remove transcriptomic responses with less than 100 DEGs
for index in tqdm(transcriptomic_responses_df.index.values):
    if sum(abs(transcriptomic_responses_df.loc[index])) < 100:
        transcriptomic_responses_df = transcriptomic_responses_df.drop(index, axis=0)

100%|██████████| 8781/8781 [01:05<00:00, 134.90it/s]


In [8]:
import os
os.getcwd()

'/Users/danieldomingo/Downloads/transcriptomic-target-correlation/notebooks'

# Get Cell Lines

In [9]:
# In order to get the data for this step download 
# Information_for_transcriptional_responses.csv from https://chempert.uni.lu/downloads
response_metdata_df = pd.read_csv(
    '../data/ChemPert_data/Information_for_transcriptional_responses.csv',
    usecols=[
        'Response_ID',
        'Chemical_ID',
        'Concentration',
        'Cell_Source'
    ],
    index_col='Response_ID',
)

In [10]:
response_metdata_df.head(1)

,Chemical_ID,Cell_Source,Concentration
Response_ID,,,
RID00001,CID01777,Hepatocyte,10uM


In [11]:
#Get ChemPert target vectors
targets_df = pd.read_csv(
    '../data/target_data_frames/target_vectors_ChemPert.tsv',
    sep='\t',
    index_col=0,
)


In [12]:
targets_df.head(1)

,AAAS,AADAC,AADACL2,AADAT,AASS,AATF,ABAT,ABCA1,ABCB1,ABCB11,...,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN5DP,ZSCAN9,ZXDA,ZXDB,ZXDC,ZZZ3
CID00001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#Create dictionary of cell lines for each drug
targets_to_cell_lines_dict = defaultdict(lambda: defaultdict(set))

for drug in tqdm(targets_df.index.values):
    current_cell_lines = [x.split('_')[1] for x in transcriptomic_responses_df.index.values if x.startswith(drug)]

    # Get this row of the target dataframe
    current_targets = targets_df.loc[drug]
    # Get all the targets that are not 0
    current_targets = list(current_targets[current_targets != 0].index)

    for target in current_targets:
        for cell_line in np.unique(current_cell_lines):
            targets_to_cell_lines_dict[target][cell_line].add(drug)


100%|██████████| 2152/2152 [00:04<00:00, 519.85it/s]


Number of unique targets

In [14]:
len(targets_to_cell_lines_dict.keys())

1606

### Load silencing experiments

Download from http://www.licpathway.net/KnockTF/index.html
Link:  http://www.licpathway.net/KnockTF/download_FC/differential%20expression%20of%20genes%20in%20all%20datasets.txt

In [15]:
silencing_experiments = pd.read_csv(
    # http://www.licpathway.net/KnockTF/download_FC/differential%20expression%20of%20genes%20in%20all%20datasets.txt
    '/Users/danieldomingo/Downloads/database.txt',
    sep='\t',
)

In [16]:
silencing_experiments.head(1)

,Sample_ID,TF,Gene,Mean Expr. of Treat,Mean Expr. of Control,FC,Log2FC,Rank,P_value,up_down
0,DataSet_01_01,ESR1,KRT4,2753.89913,90.77417,30.33795,4.92305,1,1.8795e-09,1


In [17]:
len(silencing_experiments.TF.unique())

308

Targets for which there are silencing experiments

In [18]:
valid_targets = set(targets_to_cell_lines_dict.keys()).intersection(set(silencing_experiments.TF.unique()))

Target overlap

In [19]:
len(valid_targets)

47

After manual checking these are the experiments for specific targets that are measured cell lines

In [20]:
targets_with_transcriptomics_in_the_same_cell_line = {
    # 'TP53'
    'TP53': {
        'DataSet_01_31', # HEK293T
        'DataSet_01_37', # SKB
    },
    'ATM': {
        'DataSet_01_219'  # MCF10A
    },
    'RELA': {
        'DataSet_01_30' # HEK293 / HEK293T
    },
    'PTEN': {
        'DataSet_01_303'  # SKB
    }
}

In [21]:
pairs_to_investigate = []

for target in targets_to_cell_lines_dict:

    if target not in targets_with_transcriptomics_in_the_same_cell_line:
        continue

    # Drugs that have transcriptomics in the same cell line
    if target == 'RELA':
        pairs_to_investigate.append((
            list(targets_to_cell_lines_dict[target]['HEK293T']),
            'HEK293T',
            list(targets_with_transcriptomics_in_the_same_cell_line[target])
        ))

    # """The drugs targeting TP53 have multiple other targets so it is not a one to one comparison"""
    # elif target == 'TP53':
    #     pairs_to_investigate.append((
    #         list(targets_to_cell_lines_dict[target]['HEK293T']),
    #         'HEK293T',
    #         list(targets_with_transcriptomics_in_the_same_cell_line[target])
    #     ))
    #     pairs_to_investigate.append((
    #         list(targets_to_cell_lines_dict[target]['SKB']),
    #         'SKB',
    #         list(targets_with_transcriptomics_in_the_same_cell_line[target])
    #     ))

    elif target == 'ATM':
        pairs_to_investigate.append((
            list(targets_to_cell_lines_dict[target]['MCF10A']),
            'MCF10A',
            list(targets_with_transcriptomics_in_the_same_cell_line[target])
        ))
        
    elif target == 'PTEN':
        pairs_to_investigate.append((
            list(targets_to_cell_lines_dict[target]['SKB']),
            'SKB',
            list(targets_with_transcriptomics_in_the_same_cell_line[target])
        ))


Pairs of drugs to calculate the correlation
(Drug, Cell line, Dataset from the silencing database)

In [23]:
pairs_to_investigate

[(['CID52564'], 'HEK293T', ['DataSet_01_30']),
 (['CID01466', 'CID01467', 'CID55186'], 'MCF10A', ['DataSet_01_219']),
 (['CID51684', 'CID56867'], 'SKB', ['DataSet_01_303'])]

In [24]:

for compound, cell_line, dataset in pairs_to_investigate:

    # Get the transcriptomic response for the compound
    transcriptomic_response = transcriptomic_responses_df.loc[compound[0] + '_' + cell_line]

    # silencing experiments
    silencing_subset_df = silencing_experiments[
        (silencing_experiments.Sample_ID == dataset[0])
    ]

    if len(silencing_subset_df.TF.unique()) > 2:
        raise ValueError('More than 2 genes were silenced in the experiment')

    target = silencing_subset_df.TF.unique()[0]

    # Filter by 0.05 p value
    silencing_subset_df = silencing_subset_df[silencing_subset_df.P_value < 0.05]
    
    # Get the gene names with 2 in the up_down column
    down_regulated_genes = [
        gene
        for gene in silencing_subset_df[silencing_subset_df.up_down == 2].Gene.unique()
        if gene in transcriptomic_response.index
    ]

    up_regulated_genes = [
        gene
        for gene in silencing_subset_df[silencing_subset_df.up_down == 1].Gene.unique()
        if gene in transcriptomic_response.index
    ]

    # Make a empty Series dataframe with the indexes of transcriptomic_response.index
    silencing_experiment_degs = pd.Series(index=transcriptomic_response.index)
    silencing_experiment_degs.fillna(0, inplace=True)

    # Add +1 to the up regulated genes in the silencing_experiment_degs if the index exist
    silencing_experiment_degs[up_regulated_genes] = 1
    silencing_experiment_degs[down_regulated_genes] = -1

    # Get the correlation between the target vector and the transcriptomic response
    correlation = stats.pearsonr(transcriptomic_response, silencing_experiment_degs)[0]

    print(f'Correlation between the target vector and the transcriptomic response for {compound[0]} and {target} in {cell_line} is {correlation}')


Correlation between the target vector and the transcriptomic response for CID52564 and RELA in HEK293T is -0.015310912258553721
Correlation between the target vector and the transcriptomic response for CID01466 and ATM in MCF10A is 0.03578116285119528
Correlation between the target vector and the transcriptomic response for CID51684 and PTEN in SKB is -0.013626536370243476
